In [1]:
# This script computes the completeness for the GCN2008
import pandas as pd
import os
import sys
sys.path.append('..')
import completeness as cplt

# load module defination and GCN
input_df = pd.read_csv('../data/module_def0507.tsv', sep='\t', header=0, index_col=0)
df = pd.read_csv('../data/gcn2008.tsv', sep='\t', header=0, index_col=0)

In [2]:
out_df = pd.DataFrame(index=df.index,columns=input_df.index) 
for index, row in df.iterrows():
  # print(index)
  ko_list = []
  for col, value in row.items():
    #print(col,value)
    if value != 0:
      ko_list.append(col)
  if not ko_list:  
    continue

  for idx in input_df.index:
    definition = input_df.loc[idx, 'def']
    node_dict, nid = cplt.replace_branket(definition)
    definition, mdict = cplt.replace_module(input_df, definition)
    new_node_dict, nid = cplt.replace_branket(definition, nid)
    node_dict = {**mdict, **node_dict, **new_node_dict}
    path = cplt.main(definition,ko_list, node_dict)
    value = path['exist']/path['length']
    value = round(value, 4)
    out_df.loc[index, idx] = value
    
odir = '../result/completeness'
if not os.path.exists(odir):
  os.makedirs(odir)
out_df.to_csv(os.path.join(odir, 'genome_module.completeness.tsv'), sep='\t')
